In [ ]:
from openai import OpenAI
import pandas as pd
from docx import Document

In [ ]:
client = OpenAI(api_key="your-api-key-here")

In [ ]:
def extract_text_from_docx(docx_path):
    doc = Document(docx_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)

def extract_requirements_from_text(text):
    prompt = f"""
    You are an AI assistant for business analysis. Please extract and list all functional requirements from the following document:

    {text}

    Provide a list of requirements in a clear and concise format.
    """
    response = openai.ChatCompletion.create(
        model="<model>",
        messages=[
            {"role": "system", "content": "You are a business analysis assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=1000,
        temperature=0.5
    )
    return response.choices[0].message.content

In [ ]:
def extract_requirements_from_docx(docx_path):
    # Step 1: Extract text from .docx file
    document_text = extract_text_from_docx(docx_path)
    
    # Step 2: Use model to process the text and extract requirements
    requirements = extract_requirements_from_text(document_text)
    
    return requirements

In [ ]:
def generate_test_cases(requirement):
    """
    Generate test cases from a single requirement using OpenAI.
    :param requirement: A functional requirement as a string.
    :return: Generated test cases as a string.
    """
    prompt = f"""
    Based on the following business requirement, generate detailed test cases.

    Requirement: {requirement}

    Test Case Format:
    - Test Case ID
    - Description
    - Preconditions
    - Test Steps
    - Expected Outcome
    - Priority

    Generate at least 3 test cases covering positive, negative, and edge scenarios.
    """
    
    # Use environment variable or replace with your actual key    
    try:
        response = client.chat.completions.create(
            model="<model>",
            messages=[
                {"role": "system", "content": "You are a test case generator."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=500,
            temperature=0.7
        )
        
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error generating test cases: {e}")
        return f"Error: Unable to generate test cases. {str(e)}"

In [ ]:
def export_test_cases_to_excel(test_cases, output_file):
    """
    Export test cases to an Excel file.
    :param test_cases: List of dictionaries representing test cases.
    :param output_file: Path to the output Excel file.
    """
    df = pd.DataFrame(test_cases)
    df.to_excel(output_file, index=False)
    print(f"Test cases exported to {output_file}")

In [ ]:
def process_brd_and_generate_test_cases(input_docx, output_excel):
    # Step 1: Extract requirements from BRD
    requirements = extract_requirements_from_docx(input_docx)
    
    # Step 2: Generate test cases for each requirement
    all_test_cases = []
    for i, requirement in enumerate(requirements):
        print(f"Processing requirement {i+1}/{len(requirements)}: {requirement}")
        generated_test_cases = generate_test_cases(requirement)
        all_test_cases.append({
            "Requirement": requirement,
            "Test Cases": generated_test_cases
        })
    
    # Step 3: Export test cases to Excel
    export_test_cases_to_excel(all_test_cases, output_excel)

In [ ]:
input_brd_path = "BRD_Sample.docx"  # Replace with your BRD file path
output_excel_path = "Generated_Test_Cases.xlsx"
process_brd_and_generate_test_cases(input_brd_path, output_excel_path)